<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/2_8_4__Agenci%2BSkrobaj%C5%81eba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install -q openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.24.0 requires anyio<4,>=3.1.0, but you have anyio 4.7.0 which is incompatible.


Kod instaluje dwie konkretne biblioteki Pythona:

1. openai w wersji 1.55.3 - jest to oficjalna biblioteka do komunikacji z API OpenAI
2. httpx w wersji 0.27.2 - to klient HTTP dla Pythona, który wspiera nowoczesne protokoły i standardy

Flagi użyte w poleceniu instalacyjnym:
- `-q` lub `--quiet` - wycisza większość komunikatów podczas instalacji
- `--force-reinstall` - wymusza ponowną instalację, nawet jeśli pakiety są już zainstalowane
- `--quiet` - dodatkowe wyciszenie komunikatów (dubel z `-q`)

Użycie znaku wykrzyknika na początku (`!pip`) wskazuje, że kod jest wykonywany w środowisku Jupyter Notebook lub Google Colab, gdzie taka składnia pozwala na uruchamianie poleceń systemowych.

Ta konfiguracja jest typowym pierwszym krokiem przy pracy z API OpenAI, gdzie biblioteka httpx służy jako zależność do obsługi połączeń sieciowych.

In [ ]:
! pip install -q autogen-agentchat~=0.2 apify-client

Ten kod instaluje dwie biblioteki Pythona:

1. autogen-agentchat w wersji około 0.2 (symbol `~=` oznacza, że może być zainstalowana wersja 0.2.x, ale nie 0.3) - jest to biblioteka do tworzenia konwersacyjnych agentów AI

2. apify-client - klient do platformy Apify, która służy do web scrapingu i automatyzacji zadań w internecie

Flagi i notacja:
- `!` na początku oznacza wykonanie polecenia systemowego w Jupyter Notebook lub Google Colab
- `-q` wycisza większość komunikatów podczas instalacji
- `~=` to operator kompatybilności wersji, który pozwala na instalację poprawek w ramach tej samej wersji minor


In [ ]:
from google.colab import userdata
import os
from apify_client import ApifyClient
from typing_extensions import Annotated

from autogen import ConversableAgent, register_function


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Ten kod importuje niezbędne biblioteki i komponenty:

1. `from google.colab import userdata` - importuje moduł do obsługi danych użytkownika w środowisku Google Colab

2. `import os` - importuje moduł do operacji systemowych, pozwala na pracę ze zmiennymi środowiskowymi i ścieżkami

3. `from apify_client import ApifyClient` - importuje klienta do platformy Apify, służącego do web scrapingu

4. `from typing_extensions import Annotated` - importuje typ Annotated z rozszerzeń typing, używany do dodawania metadanych do typów

5. `from autogen import ConversableAgent, register_function` - importuje dwa komponenty z biblioteki autogen:
   - ConversableAgent - bazową klasę do tworzenia agentów zdolnych do prowadzenia konwersacji
   - register_function - dekorator do rejestrowania funkcji, które mogą być używane przez agentów

Te importy tworzą podstawę do budowy systemu, który może łączyć funkcje agentów konwersacyjnych ze zbieraniem danych z internetu, z wykorzystaniem specyficznych funkcji Google Colab.

In [ ]:
class CFG:
  model = 'gpt-4o-mini'

In [ ]:
# configs

config_list = [
    {"model": CFG.model, "api_key": userdata.get('openaivision')},
]

apify_api_key = userdata.get('apify')

Ten fragment kodu konfiguruje kluczowe ustawienia:

1. `config_list` to lista zawierająca słownik z dwiema wartościami:
   - `model` - pobiera model z obiektu CFG (zdefiniowanego wcześniej)
   - `api_key` - pobiera klucz API do OpenAI Vision z danych użytkownika Colab

2. `apify_api_key` - zapisuje w zmiennej klucz API do platformy Apify, również pobrany z danych użytkownika Colab

W obu przypadkach użyto metody `userdata.get()`, która bezpiecznie pobiera poufne dane uwierzytelniające przechowywane w środowisku Google Colab. Ten sposób przechowywania kluczy API jest bezpieczniejszy niż umieszczanie ich bezpośrednio w kodzie.

Ta konfiguracja przygotowuje środowisko do pracy zarówno z modelami wizyjnymi OpenAI, jak i usługami Apify.

# Functions

In [ ]:
def scrape_page(url: Annotated[str, "The URL of the web page to scrape"]) -> Annotated[str, "Scraped content"]:
    # Initialize the ApifyClient with your API token
    client = ApifyClient(token=apify_api_key)

    # Prepare the Actor input
    run_input = {
        "startUrls": [{"url": url}],
        "useSitemaps": False,
        "crawlerType": "playwright:firefox",
        "includeUrlGlobs": [],
        "excludeUrlGlobs": [],
        "ignoreCanonicalUrl": False,
        "maxCrawlDepth": 0,
        "maxCrawlPages": 1,
        "initialConcurrency": 0,
        "maxConcurrency": 200,
        "initialCookies": [],
        "proxyConfiguration": {"useApifyProxy": True},
        "maxSessionRotations": 10,
        "maxRequestRetries": 5,
        "requestTimeoutSecs": 60,
        "dynamicContentWaitSecs": 10,
        "maxScrollHeightPixels": 5000,
        "removeElementsCssSelector": """nav, footer, script, style, noscript, svg,
    [role=\"alert\"],
    [role=\"banner\"],
    [role=\"dialog\"],
    [role=\"alertdialog\"],
    [role=\"region\"][aria-label*=\"skip\" i],
    [aria-modal=\"true\"]""",
        "removeCookieWarnings": True,
        "clickElementsCssSelector": '[aria-expanded="false"]',
        "htmlTransformer": "readableText",
        "readableTextCharThreshold": 100,
        "aggressivePrune": False,
        "debugMode": True,
        "debugLog": True,
        "saveHtml": True,
        "saveMarkdown": True,
        "saveFiles": False,
        "saveScreenshots": False,
        "maxResults": 9999999,
        "clientSideMinChangePercentage": 15,
        "renderingTypeDetectionPercentage": 10,
    }

    # Run the Actor and wait for it to finish
    run = client.actor("aYG0l9s7dbB7j3gbS").call(run_input=run_input)

    # Fetch and print Actor results from the run's dataset (if there are any)
    text_data = ""
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        text_data += item.get("text", "") + "\n"

    average_token = 0.75
    max_tokens = 20000  # slightly less than max to be safe 32k
    text_data = text_data[: int(average_token * max_tokens)]
    return text_data

Ta funkcja `scrape_page` służy do zaawansowanego scrapowania stron internetowych:

1. Przyjmuje parametr `url` (z adnotacją typu wskazującą, że to adres strony do scrapowania) i zwraca wydobytą treść tekstową.

2. Inicjalizuje klienta Apify używając wcześniej skonfigurowanego klucza API.

3. Konfiguruje szczegółowe parametry scrapowania w `run_input`:
   - Ogranicza crawling do jednej strony (`maxCrawlPages`: 1)
   - Używa przeglądarki Firefox przez Playwright
   - Ustawia proxy Apify
   - Definiuje maksymalny czas oczekiwania i próby ponowienia
   - Usuwa zbędne elementy strony (nawigację, stopkę, skrypty)
   - Usuwa ostrzeżenia o ciasteczkach
   - Rozwija zwinięte elementy
   - Zapisuje wyniki w formacie HTML i Markdown

4. Uruchamia aktora Apify o ID "aYG0l9s7dbB7j3gbS" z tymi ustawieniami.

5. Zbiera wyniki z datasetu:
   - Łączy wszystkie znalezione teksty
   - Ogranicza długość wyniku do około 20000 tokenów (używając współczynnika 0.75 tokena na znak)

Funkcja jest zoptymalizowana pod kątem czytelności wyników i obsługi dynamicznych stron internetowych.

# Agents

In [ ]:

scraper_agent = ConversableAgent(
    "WebScraper",
    llm_config={"config_list": config_list},
    system_message="You are a web scrapper and you can scrape any web page using the tools provided. "
    "Returns 'TERMINATE' when the scraping is done.",
)



Ten kod tworzy agenta do scrapowania stron (`scraper_agent`) używając klasy `ConversableAgent`:

1. Nadaje mu nazwę "WebScraper"

2. Konfiguruje model językowy poprzez `llm_config`, który używa wcześniej zdefiniowanej listy `config_list` z ustawieniami modelu i kluczem API

3. Definiuje wiadomość systemową określającą rolę i zachowanie agenta:
   - Agent identyfikuje się jako web scraper
   - Ma możliwość scrapowania dowolnej strony przy użyciu dostarczonych narzędzi
   - Zwraca 'TERMINATE' po zakończeniu scrapowania

Ten agent będzie mógł prowadzić konwersację i wykonywać zadania scrapowania w odpowiedzi na polecenia użytkownika.

In [ ]:
user_proxy_agent = ConversableAgent(
    "UserProxy",
    llm_config=False,
    human_input_mode="NEVER",
    code_execution_config=False,
    is_termination_msg=lambda x: x.get("content", "") is not None and "terminate" in x["content"].lower(),
    default_auto_reply="Please continue if not finished, otherwise return 'TERMINATE'.",
)

Ten kod tworzy agenta reprezentującego użytkownika (`user_proxy_agent`) z następującymi ustawieniami:

1. Nazwa agenta to "UserProxy"

2. `llm_config=False` - wyłącza model językowy dla tego agenta, co oznacza że nie będzie generował własnych odpowiedzi

3. `human_input_mode="NEVER"` - agent nigdy nie będzie prosił o input od człowieka

4. `code_execution_config=False` - wyłącza możliwość wykonywania kodu przez tego agenta

5. Definiuje funkcję sprawdzającą warunek zakończenia:
   - `is_termination_msg=lambda x: x.get("content", "") is not None and "terminate" in x["content"].lower()`
   - Sprawdza czy wiadomość zawiera słowo "terminate" (niezależnie od wielkości liter)

6. `default_auto_reply` ustawia domyślną odpowiedź:
   - "Please continue if not finished, otherwise return 'TERMINATE'."
   - Ta wiadomość jest wysyłana automatycznie gdy agent nie ma innych instrukcji

Ten agent służy jako pośrednik między systemem a prawdziwym użytkownikiem, automatyzując interakcje bez potrzeby ręcznego wprowadzania danych.

In [ ]:
register_function(
    scrape_page,
    caller=scraper_agent,
    executor=user_proxy_agent,
    name="scrape_page",
    description="Scrape a web page and return the content.",
)


Ten kod rejestruje funkcję `scrape_page` w systemie agentów:

1. `scrape_page` - rejestrowana funkcja do scrapowania stron

2. `caller=scraper_agent` - określa, że scraper_agent może wywoływać tę funkcję

3. `executor=user_proxy_agent` - wskazuje, że user_proxy_agent będzie wykonywał tę funkcję

4. `name="scrape_page"` - definiuje nazwę, pod którą funkcja będzie dostępna

5. `description="Scrape a web page and return the content."` - dodaje opis funkcjonalności

Ta rejestracja umożliwia współpracę między agentami - scraper_agent może zlecać zadania scrapowania, które będą wykonywane przez user_proxy_agent.

# Run

In [ ]:
chat_result = user_proxy_agent.initiate_chat(
    scraper_agent,
    message="Can you scrape agentops.ai for me?",
    summary_method="reflection_with_llm",
    summary_args={
        "summary_prompt": """Summarize the scraped content and format summary EXACTLY as follows:
---
*Company name*:
`Acme Corp`
---
*Website*:
`acmecorp.com`
---
*Description*:
`Company that does things.`
---
*Tags*:
`Manufacturing. Retail. E-commerce.`
---
*Takeaways*:
`Provides shareholders with value by selling products.`
---
*Questions*:
`What products do they sell? How do they make money? What is their market share?`
---
"""
    },
)

UserProxy (to WebScraper):

Can you scrape agentops.ai for me?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
WebScraper (to UserProxy):

***** Suggested tool call (call_AmEdRdxwobx4rKwofvFKsTLE): scrape_page *****
Arguments: 
{"url":"https://agentops.ai"}
****************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION scrape_page...
UserProxy (to WebScraper):

UserProxy (to WebScraper):

***** Response from calling tool (call_AmEdRdxwobx4rKwofvFKsTLE) *****
AgentOps
Every Agent
Needs AgentOps.
Industry leading developer platform to test and debug AI agents. 
We built the tools so you don't have to.
Powering thousands of engineers building reliable agents
Action Research agent
Update DB
Updated successfully
Timings
Start - End
19.91s - 23.38s
Visualize
Visually track events such as LLM calls, tools, 

In [ ]:
print(chat_result.summary)

---
*Company name*:
`AgentOps`
---
*Website*:
`agentops.ai`
---
*Description*:
`Industry-leading developer platform to test and debug AI agents.`
---
*Tags*:
`AI. Development. Debugging. Cost Management.`
---
*Takeaways*:
`Offers tools for visualizing agent interactions, time travel debugging, and comprehensive cost tracking.`
---
*Questions*:
`What specific AI agents can be built with AgentOps? How does the pricing model scale with usage? What integrated frameworks are supported?`
---


Ten kod inicjuje rozmowę między agentami w celu zebrania i analizy danych o stronie agentops.ai:

1. `user_proxy_agent.initiate_chat()` rozpoczyna konwersację z następującymi parametrami:

2. Partnerem rozmowy jest `scraper_agent`

3. Wiadomość początkowa to prośba o scrapowanie agentops.ai

4. Metoda podsumowania jest ustawiona na "reflection_with_llm" - używa modelu językowego do analizy

5. `summary_args` definiuje format podsumowania z dokładną strukturą:
   - Nazwa firmy
   - Strona internetowa
   - Opis
   - Tagi
   - Główne wnioski
   - Pytania do dalszej analizy

Każda sekcja w podsumowaniu jest oznaczona markdown'em z gwiazdkami dla nagłówków i znakami backtick dla treści.

Ten kod nie tylko zbierze dane ze strony, ale również automatycznie je przeanalizuje i sformatuje według podanego szablonu.